# End-to-End NLP: News Headline Classifier (SageMaker Version)

_**Train a PyTorch-based model to classify news headlines between four domains**_

This notebook works well with the `Python 3 (PyTorch 1.13 Python 3.9 CPU Optimized)` kernel on SageMaker Studio, or `conda_pytorch_p38` on classic SageMaker Notebook Instances.

---

Following on from the previous 'local version' notebook, we show here how to trigger the model training and deployment on separate infrastructure - to make better use of resources.

Note that you can safely ignore the WARNING about the pip version.

In [ ]:
# First install some libraries which might not be available across all kernels (e.g. in Studio):
!pip install "ipywidgets<8" torchtext==0.6

### Download News Aggregator Dataset

We will download **FastAi AG News** dataset from the [Registry of Open Data on AWS](https://registry.opendata.aws/fast-ai-nlp/) public repository. This dataset contains a table of news headlines and their corresponding classes.


In [ ]:
%%time
local_dir = "data"
# Download the AG News data from the Registry of Open Data on AWS.
!mkdir -p {local_dir}
!aws s3 cp s3://fast-ai-nlp/ag_news_csv.tgz {local_dir} --no-sign-request

# Un-tar the AG News data.
!tar zxf {local_dir}/ag_news_csv.tgz -C {local_dir}/ --strip-components=1 --no-same-owner
print("Done!")

### Let's visualize the dataset

We will load the ag_news_csv/train.csv file to a Pandas dataframe for our data processing work.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import re

import numpy as np
import pandas as pd
import util.preprocessing

In [ ]:
column_names = ["CATEGORY", "TITLE", "CONTENT"]
# we use the train.csv only
df = pd.read_csv(f"{local_dir}/train.csv", names=column_names, header=None, delimiter=",")
# shuffle the DataFrame rows
df = df.sample(frac=1, random_state=1337)
# make the category classes more readable
mapping = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tech"}
df = df.replace({"CATEGORY": mapping})
df.head()

For this exercise we'll **only use**:

- The **title** (Headline) of the news story, as our input
- The **category**, as our target variable


In [ ]:
df["CATEGORY"].value_counts()

The dataset has **four article categories** with equal weighting:

- Business
- Sci/Tech
- Sports
- World


## Natural Language Pre-Processing

We'll do some basic processing of the text data to convert it into numerical form that the algorithm will be able to consume to create a model.

We will do typical pre processing for NLP workloads such as: dummy encoding the labels, tokenizing the documents and set fixed sequence lengths for input feature dimension, padding documents to have fixed length input vectors.


### Dummy Encode the Labels


In [ ]:
encoded_y, labels = util.preprocessing.dummy_encode_labels(df, "CATEGORY")
print(labels)
print(encoded_y)

For example, looking at the first record in our (shuffled) dataframe:

In [ ]:
df["CATEGORY"].iloc[0]

In [ ]:
encoded_y[0]

### Tokenize and Set Fixed Sequence Lengths

We want to describe our inputs at the more meaningful word level (rather than individual characters), and ensure a fixed length of the input feature dimension.


In [ ]:
processed_docs, tokenizer = util.preprocessing.tokenize_and_pad_docs(df, "TITLE")

In [ ]:
df["TITLE"].iloc[0]

In [ ]:
processed_docs[0]

### Import Word Embeddings

To represent our words in numeric form, we'll use pre-trained vector representations for each word in the vocabulary: In this case we'll be using [pre-trained word embeddings from FastText](https://fasttext.cc/docs/en/crawl-vectors.html), which are also available for a broad range of languages other than English.

You could also explore training custom, domain-specific word embeddings using SageMaker's built-in [BlazingText algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html). See the official [blazingtext_word2vec_text8 sample](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/introduction_to_amazon_algorithms/blazingtext_word2vec_text8) for an example notebook showing how.

> ⚠️ You may sometimes see a file format error if you run this cell at the same time as the `get_word_embeddings()` cell in the local notebook. If this happens, try deleting the `data/embeddings` folder before re-running this cell.

In [ ]:
%%time
embedding_matrix = util.preprocessing.get_word_embeddings(tokenizer, "data/embeddings")

In [ ]:
np.save(
    file=f"{local_dir}/embeddings/docs-embedding-matrix",
    arr=embedding_matrix,
    allow_pickle=False,
)
vocab_size = embedding_matrix.shape[0]
print(embedding_matrix.shape)

### Split Train and Test Sets

Finally we need to divide our data into model training and evaluation sets:


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    processed_docs,
    encoded_y,
    test_size=0.2,
    random_state=42,
)

In [ ]:
os.makedirs(f"{local_dir}/train", exist_ok=True)
np.save(f"{local_dir}/train/train_X.npy", X_train)
np.save(f"{local_dir}/train/train_Y.npy", y_train)
os.makedirs(f"{local_dir}/test", exist_ok=True)
np.save(f"{local_dir}/test/test_X.npy", X_test)
np.save(f"{local_dir}/test/test_Y.npy", y_test)

### Set Up Execution Role, Session and S3 Bucket

The primary data source for a SageMaker training job is (nearly) always Amazon S3. We'll need to upload our pre-processed data there to make it available for SageMaker training jobs.

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting. If you don't specify a bucket, SageMaker SDK will create a default bucket following a pre-defined naming convention in the same region. 
- The IAM role ARN used to give SageMaker access to your data. It can be fetched using the **get_execution_role** method from sagemaker python SDK.

In [ ]:
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()
bucket_name = sess.default_bucket()

## Upload Data to Amazon S3

One way to upload data to S3 is by using the high-level [aws s3 sync](https://docs.aws.amazon.com/cli/latest/userguide/cli-services-s3-commands.html#using-s3-commands-managing-objects-sync) command that synchronizes the contents of the target bucket and source directory. This command also supports options such as:

- `--delete`, to remove any objects from the target that aren't present in the source
- `--exclude` and `--include`, to filter files/objects instead of copying everything from the source folder.

Although it's possible to work with S3 via Python as well (for example with [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html) or the [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/api/utility/s3.html)), the S3 sync CLI is also natively *multi-threaded*: Which helps deliver fast transfers from notebooks without introducing more complex code.


In [ ]:
!aws s3 sync --quiet --delete {local_dir} s3://{bucket_name}/news --exclude "*" --include "*.npy"
print("Done!")

## Data Input ("Channels") Configuration

The local version of the notebook ([Headline Classifier Local.ipynb](https://github.com/aws-samples/sagemaker-101-workshop/blob/d49b827f647c9f050879ca60e81afc890d0e7a22/custom_tensorflow_keras_nlp/Headline%20Classifier%20Local.ipynb)) has three data inputs -- train, test and embeddings. In Sagemaker terminology, each input data is a "channel".

In [ ]:
train_channel = f"s3://{bucket_name}/news/train"
test_channel = f"s3://{bucket_name}/news/test"
embeddings_channel = f"s3://{bucket_name}/news/embeddings"

inputs = {"train": train_channel, "test": test_channel, "embeddings": embeddings_channel}
print(inputs)

## Train with Differentiated Infrastructure on Sagemaker

This time, we've packaged the model build and train code from our previous notebook ([Headline Classifier Local.ipynb](Headline%20Classifier%20Local.ipynb)) into the [**main.py**](src/main.py) script in the **src** directory.

We'll use the high-level [PyTorch Framework Container through the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html) to train and deploy the model from this script file.

### How Amazon SageMaker runs your PyTorch script with pre-built containers

AWS provides a pre-packaged set of Docker images to help you accelerate building your projects on major ML frameworks: The [SageMaker Framework Containers](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers-prebuilt.html).

These containers take care of basic setup like GPU drivers, serving stack implementation, core libraries, and so on - leaving us free to simply inject some Python scripts for the training process and any inference behaviour overrides. We can even provide a *requirements.txt* file to specify additional dependencies to be dynamically installed at start-up - without having to build these into the container image.

**As a result, our first task is to understand the interfaces** between our script(s) and the runtime: How will the script read input data? Parameters? Where should it store results?


#### Running your container during training

When Amazon SageMaker runs training, your training script (entry_point input) is run just like a regular Python program. A number of files are laid out for your use, under a `/opt/ml` directory. These will be locations that you can access from within your script. You will see an example of the use of this in our [**main.py**](src/main.py):

    /opt/ml
    |-- code
    |   `-- <our script(s)>
    |-- input
    |   |-- config
    |   |   |-- hyperparameters.json
    |   |   `-- resourceConfig.json
    |   `-- data
    |       `-- <channel_name>
    |           `-- <input data>
    |-- model
    |   `-- <model files>
    `-- output
        `-- failure

##### The input

* `/opt/ml/input/config` contains information to control how your program runs. `hyperparameters.json` is a JSON-formatted dictionary of hyperparameter names to values. These values will always be strings, so you may need to convert them. `resourceConfig.json` is a JSON-formatted file that describes the network layout used for distributed training. Since scikit-learn doesn't support distributed training, we'll ignore it here.
* `/opt/ml/input/data/<channel_name>/` (for File mode) contains the input data for that channel. The channels are created based on the call to CreateTrainingJob but it's generally important that channels match what the algorithm expects. The files for each channel will be copied from S3 to this directory, preserving the tree structure indicated by the S3 key structure. 
* `/opt/ml/input/data/<channel_name>_<epoch_number>` (for Pipe mode) is the pipe for a given epoch. Epochs start at zero and go up by one each time you read them. There is no limit to the number of epochs that you can run, but you must close each pipe before reading the next epoch.

##### The output

* `/opt/ml/model/` is the directory where you write the model that your algorithm generates. Your model can be in any format that you want. It can be a single file or a whole directory tree. SageMaker will package any files in this directory into a compressed tar archive file. This file will be available at the S3 location returned in the `DescribeTrainingJob` result.
* `/opt/ml/output` is a directory where the algorithm can write a file `failure` that describes why the job failed. The contents of this file will be returned in the `FailureReason` field of the `DescribeTrainingJob` result. For jobs that succeed, there is no reason to write this file as it will be ignored.

#### Further information

For more information, you can refer to:

- The [SageMaker Python SDK guide for PyTorch](https://sagemaker.readthedocs.io/en/stable/using_pytorch.html) and [API doc](https://sagemaker.readthedocs.io/en/stable/sagemaker.pytorch.html) for PyTorch framework classes.
- The [AWS Deep Learning Containers repository](https://github.com/aws/deep-learning-containers) on GitHub, which defines the underlying container images.
- The open source SageMaker [PyTorch Training Toolkit](https://github.com/aws/sagemaker-pytorch-training-toolkit) and [PyTorch Inference Toolkit](https://github.com/aws/sagemaker-pytorch-inference-toolkit) for more details on the framework code for training and serving.


In [ ]:
import sagemaker
from sagemaker.pytorch.estimator import PyTorch as PyTorchEstimator

Although the script will run on a separate container, we can pass whatever parameters it needs through SageMaker:


In [ ]:
hyperparameters = {"epochs": 5, "num_classes": 4}

We have our `PyTorch` estimator object, we have set the hyper-parameters for this object and we have our data channels linked with the algorithm. The only remaining thing to do is to train the algorithm. The following command will run this training, which involves a few steps:

- Firstly, the instance that we requested while creating the `TensorFlow` estimator classes is provisioned and is setup with the appropriate libraries.
- Then, the data from our channels are downloaded into the instance.
- Once this is done, the training job begins running your code.

The provisioning and data downloading will take some time, depending on the size of the data and the container image. 

Once the job has finished a "Job complete" message will be printed. The trained model can be found in the S3 bucket that was setup as `output_path` in the estimator.

Here we run the training job using the compute-optimized (CPU) `ml.c5.xlarge` instance type. For larger deep learning models or datasets, you may want to use GPU-accelerated types like `ml.p*` or `ml.g*` to speed up training further.

In [ ]:
%%time

metric_definitions = [
    {"Name": "Epoch", "Regex": r"epoch: ([0-9\.]+)"},
    {"Name": "Train:Loss", "Regex": r"train_loss: ([0-9\.]+)"},
    {"Name": "Validation:Loss", "Regex": r"val_loss: ([0-9\.]+)"},
    {"Name": "Validation:Accuracy", "Regex": r"val_acc: ([0-9\.]+)"},
]

estimator = PyTorchEstimator(
    framework_version="1.10",
    py_version="py38",
    instance_count=1,
    instance_type="ml.c5.xlarge",
    role=role,
    entry_point="main.py",
    source_dir="./src",
    hyperparameters=hyperparameters,
    metric_definitions=metric_definitions,
    base_job_name="news-pytorch",
    max_run=20 * 60,  # Maximum allowed active runtime
    ## Here Spot is left OFF by default to avoid delays, but easy to turn on by un-commenting:
    # use_spot_instances=True,  # Use spot instances to reduce cost
    # max_wait=30*60,  # Maximum clock time (including spot wait time)
)

estimator.fit(inputs)

## Use the Model: Hosting / Inference

Once the training is done, we can deploy the trained model as an Amazon SageMaker hosted [real-time inference endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html). This will allow us to query predictions (or inferences) from the model as-needed.

Note that:

- For batch inference situations (such as calculating metrics on a test dataset), it may be advisable to use [SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) instead. See [Using SageMaker Batch Transform](https://sagemaker.readthedocs.io/en/stable/overview.html#sagemaker-batch-transform) in the SageMaker Python SDK docs for more details.
- We _don't need to use the same **type**_ of instance that we trained on. Since this test endpoint will handle very little traffic - we can go ahead and use a cheaper type:


In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
)

### Your model should now be in production as a RESTful API!

Let's evaluate our model with some example headlines...

If you struggle with the widget, you can always simply call the `classify()` function from Python.

You can be creative with your headlines!


In [ ]:
import ipywidgets as widgets
import torch
from IPython import display


def classify(text):
    """Classify a headline and print the results"""
    processed = tokenizer.preprocess(text)
    # The tokenizer and the model expect batches (lists) of multiple texts at once:
    padded = tokenizer.pad([processed])
    word_ids = [[tokenizer.vocab.stoi[w] for w in sentence] for sentence in padded]
    result = predictor.predict(word_ids)
    print(result)
    ix = np.argmax(result)
    print(f"Predicted class: '{labels[ix]}' with confidence {result[0][ix]:.2%}")


# Either try out the interactive widget:
interaction = widgets.interact_manual(
    classify,
    text=widgets.Text(
        value="The markets were bullish after news of the merger",
        placeholder="Type a news headline...",
        description="Headline:",
        layout=widgets.Layout(width="99%"),
    ),
)
interaction.widget.children[1].description = "Classify!"

In [ ]:
# Or just use the function to classify your own headline:
classify("Retailers are expanding after the recent economic growth")

## Clean up

Unlike training jobs (which destroy their resources as soon as training is finished), real-time endpoint deployments provision instances until we specifically shut the endpoint down...

So let's be frugal with resources, and delete resources when we don't need them anymore. You can un-comment and run the cell below to delete the endpoint:

In [ ]:
# predictor.delete_endpoint(delete_endpoint_config=True)

## (Optional) Automatic Hyperparameter Optimization - HPO

Rather than manually tweak parameters to tune the model performance, we can get SageMaker to help us out.

We'll simply tell SageMaker:

- The type and allowable range of each parameter,
- The metric we want to optimize for, and
- Strategy and resource constraints

...and the service will set up jobs for us to find the best combination.


### (Hyper-)Parameter Definitions


In [ ]:
from sagemaker.tuner import (
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
    IntegerParameter,
)

hyperparameter_ranges = {
    "epochs": IntegerParameter(2, 7),
    "learning_rate": ContinuousParameter(0.01, 0.2),
}

### Objective Metric

'Metrics' in SageMaker are scraped from the console output of jobs, by way of regular expressions.

We can define multiple metrics to monitor, but HPO requires us to specify that exactly one of them is the **objective** metric to optimize:


In [ ]:
# (See metric_definitions above)
objective_metric_name = "Validation:Accuracy"
objective_type = "Maximize"

### Start the Tuning Job

We already defined our Estimator above, so we'll just re-use the configuration with minor adjustments.

Note that the Estimator's `hyperparameters` will be used as base values, and overridden by the HyperParameterTuner where appropriate.


In [ ]:
# Keep per-job resources modest, so that parallel jobs don't hit any limits:
estimator.instance_type = "ml.c5.xlarge"
estimator.instance_count = 1

In [ ]:
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    base_tuning_job_name="news-hpo-pytorch",
    max_jobs=6,
    max_parallel_jobs=2,
    objective_type=objective_type,
)

tuner.fit(inputs)

### Check On Progress

HPO jobs can take a long time to complete, and can run multiple training jobs in parallel - each on multiple instances... Which is why the `fit()` call above doesn't wait by default, and won't show us a potentially-confusing consolidated log stream.

Go to the Training > Hyperparameter Tuning Jobs page of the [**SageMaker Console**](https://console.aws.amazon.com/sagemaker/home#/hyper-tuning-jobs) and select the job from the list.

You can see all the training jobs triggered for the HPO run, as well as overall summary metrics.

This information can be accessed via the API/SDKs too of course. For example we can wait for HPO to finish like the below:


In [ ]:
import time

import boto3

# Wait until HPO is finished
hpo_state = None
smclient = boto3.Session().client("sagemaker")

while hpo_state is None or hpo_state == "InProgress":
    if hpo_state is not None:
        print("-", end="")
        time.sleep(60)  # Poll once every 1 min
    hpo_state = smclient.describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
    )["HyperParameterTuningJobStatus"]

print("\nHPO state:", hpo_state)

### Using the Model

Just like with our `estimator`, we can call `tuner.deploy()` to create an endpoint and `predictor` from the best-performing model found in the HPO run.


## Review

In this notebook, we refactored our local code to train and deploy the same Keras model using SageMaker.

Some benefits of this approach are:

- We can automatically provision specialist computing resources (e.g. high-performance, or GPU-accelerated instances) for **only** the duration of the training job: Getting good performance in training, without leaving resources sitting around under-utilized
- The history of training jobs (including parameters, metrics, outputs, etc.) is automatically tracked - unlike local notebook experiments where the user needs to keep notes on what worked and what didn't
- Our trained model can be deployed to a secure, production-ready web endpoint with just one SDK call: No container or web application packaging required, unless we want to customize the behaviour
